<a href="https://colab.research.google.com/github/vithurshiniS/Twitter-Sentiment-Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/chat80.zip.
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2000.zip.
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2002.zip.
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/dependency_treebank.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    

True

Language Detection


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import string
import pickle
import nltk
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Sentiment Analysis

In [ ]:
import nltk
import random
#from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize



class VoteClassifier(ClassifierI):
  def __init__(self, *classifiers):
    self._classifiers = classifiers

  def classify(self, features):
    votes = []
    for c in self._classifiers:
      v = c.classify(features)
      votes.append(v)
    return mode(votes)

  def confidence(self, features):
    votes = []
    for c in self._classifiers:
      v = c.classify(features)
      votes.append(v)

    choice_votes = votes.count(mode(votes))
    conf = choice_votes / len(votes)
    return conf
    
short_pos = open("/content/drive/MyDrive/Research/positive_en.txt","r",encoding="latin-1").read()
short_neg = open("/content/drive/MyDrive/Research/negative_en.txt","r",encoding="latin-1").read()

# move this up here
all_words = []
documents = []


#  j is adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]

for p in short_pos.split('\n'):
  documents.append( (p, "pos") )
  words = word_tokenize(p)
  pos = nltk.pos_tag(words)
  for w in pos:
    if w[1][0] in allowed_word_types:
      all_words.append(w[0].lower())

    
for p in short_neg.split('\n'):
  documents.append( (p, "neg") )
  words = word_tokenize(p)
  pos = nltk.pos_tag(words)
  for w in pos:
    if w[1][0] in allowed_word_types:
      all_words.append(w[0].lower())



save_documents = open("/content/drive/MyDrive/Research/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()


all_words = nltk.FreqDist(all_words)


word_features = list(all_words.keys())[:5000]


save_word_features = open("/content/drive/MyDrive/Research/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()


def find_features(document):
  words = word_tokenize(document)
  features = {}
  for w in word_features:
    features[w] = (w in words)

  return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[10000:]
training_set = featuresets[:10000]


classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

###############
save_classifier = open("/content/drive/MyDrive/Research/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

save_classifier = open("/content/drive/MyDrive/Research/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

save_classifier = open("/content/drive/MyDrive/Research/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

save_classifier = open("/content/drive/MyDrive/Research/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()


LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

save_classifier = open("/content/drive/MyDrive/Research/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()


##NuSVC_classifier = SklearnClassifier(NuSVC())
##NuSVC_classifier.train(training_set)
##print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

save_classifier = open("/content/drive/MyDrive/Research/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

10664
Original Naive Bayes Algo accuracy percent: 71.3855421686747
Most Informative Features
              engrossing = True              pos : neg    =     20.9 : 1.0
                  boring = True              neg : pos    =     19.5 : 1.0
                   flaws = True              pos : neg    =     16.2 : 1.0
                mediocre = True              neg : pos    =     15.7 : 1.0
                 routine = True              neg : pos    =     15.1 : 1.0
                    flat = True              neg : pos    =     14.7 : 1.0
              refreshing = True              pos : neg    =     13.6 : 1.0
               inventive = True              pos : neg    =     13.6 : 1.0
                    warm = True              pos : neg    =     12.5 : 1.0
               wonderful = True              pos : neg    =     12.5 : 1.0
                powerful = True              pos : neg    =     12.4 : 1.0
                touching = True              pos : neg    =     11.2 : 1.0
       

Creating sentiment module


In [ ]:
#File: sentiment_mod.py

import nltk
import random
#from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize



class VoteClassifier(ClassifierI):
  def __init__(self, *classifiers):
    self._classifiers = classifiers

  def classify(self, features):
    votes = []
    for c in self._classifiers:
      v = c.classify(features)
      votes.append(v)
    return mode(votes)

  def confidence(self, features):
    votes = []
    for c in self._classifiers:
      v = c.classify(features)
      votes.append(v)

    choice_votes = votes.count(mode(votes))
    conf = choice_votes / len(votes)
    return conf


documents_f = open("/content/drive/MyDrive/Research/documents.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()




word_features5k_f = open("/content/drive/MyDrive/Research/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()


def find_features(document):
  words = word_tokenize(document)
  features = {}
  for w in word_features:
    features[w] = (w in words)

  return features



# featuresets_f = open("/content/drive/MyDrive/Research_2021/featuresets.pickle", "rb")
# featuresets = pickle.load(featuresets_f)
# featuresets_f.close()

featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[10000:]
training_set = featuresets[:10000]



open_file = open("/content/drive/MyDrive/Research/originalnaivebayes5k.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()


open_file = open("/content/drive/MyDrive/Research/MNB_classifier5k.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()



open_file = open("/content/drive/MyDrive/Research/BernoulliNB_classifier5k.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("/content/drive/MyDrive/Research/LogisticRegression_classifier5k.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()


open_file = open("/content/drive/MyDrive/Research/LinearSVC_classifier5k.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()


open_file = open("/content/drive/MyDrive/Research/SGDC_classifier5k.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()




voted_classifier = VoteClassifier(classifier,LinearSVC_classifier,MNB_classifier,BernoulliNB_classifier,LogisticRegression_classifier)




def sentiment(text):
  feats = find_features(text)
  return voted_classifier.classify(feats),voted_classifier.confidence(feats)



print(sentiment("This movie was awesome! The acting was great, plot was wonderful, and there were pythons...so yea!"))
print(sentiment("This movie was utter junk. There were absolutely 0 pythons. I don't see what the point was at all. Horrible movie, 0/10"))

10664
('pos', 1.0)
('neg', 1.0)


Importing sentiment module

In [ ]:
!ls /content/drive/MyDrive/Research/sent_mod1.py

/content/drive/MyDrive/Research/sent_mod1.py


In [ ]:
import sys

sys.path.append('/content/drive/MyDrive/Research')

In [ ]:
import sent_mod1 as s

print(s.sentiment("This movie was awesome! The acting was great, plot was wonderful, and there were pythons...so yea!"))
print(s.sentiment("This movie was utter junk. There were absolutely 0 pythons. I don't see what the point was at all. Horrible movie, 0/10"))

10664
('pos', 1.0)
('neg', 1.0)


Sentiment Analysis without language detection
Social Media Streaming

In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sent_mod1 as s

#consumer key, consumer secret, access token, access secret.
ckey="xxxxxxxxxxxxxxxxxxxxx"
csecret="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
atoken="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
asecret="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

#from twitterapistuff import *

class listener(StreamListener):
  def on_data(self, data):
    all_data = json.loads(data)
    tweet = all_data["text"]
    sentiment_value, confidence = s.sentiment(tweet)
    print(tweet, sentiment_value)

	  # if confidence*100 >= 80:
    #   output = open("twitter-out.txt","a")
    #   output.write(sentiment_value)
    #   output.write('\n')
    #   output.close()

      #return True

  def on_error(self, status):
    print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["valimai"])

RT @btsqtsarchive: #BTS In 7Fates: CHAKHO 

#ZEHA_JungKook is half tiger, half human and has a tiger power fighting against beoms alongside…
RT @WasimJaffer14: Captain, Opener, No.3, Wicket Keeper, ICC event winning coach, former NCA head, and now India coach. But above all a gre…
RT @FCM_Production: 01 Year of Master 🥺😎
@actorvijay #Master 
.
#Beast #BeastFirstSingle 
#BeastUpdate https://t.co/0Eb9HsCgTI
RT @TeaPainUSA: All over America tonight, people that think they’re part of a master race are drinkin’ their own pee.
RT @42M4rvin: Read the rest of the article! https://t.co/YvjA5C8kqd
fisting #balls #ballbusting #ballbashing #punch #big
#balls #bust #fist #hung #dick #jock #ballslick #ballbuster… https://t.co/y7n9NMCNfn
RT @TeaPainUSA: All over America tonight, people that think they’re part of a master race are drinkin’ their own pee.
RT @LuvDatta_INC: In Political Science we studied that a bunch of Ministers Sail together &amp; Sink together. Nowadays Godi Media too is inclu…
@set

KeyboardInterrupt: ignored

In [ ]:
# from tweepy import Stream
# from tweepy import OAuthHandler
# from tweepy.streaming import StreamListener
# import json
# import sent_mod1 as s

# #consumer key, consumer secret, access token, access secret.
# ckey="XXeG5cy7QUfahiOHlacBaddIF"
# csecret="quSrqvVZUfRqbp8SZ0bDZimz8avlS96htsonlG1dFAehgsmMI1"
# atoken="1399846090699067392-W14id1VCFiLIib3MD7kaT2C6pvDmvX"
# asecret="6nrOLFSELerY9HzGSV5YGNigZnhSM54izskj2iahnFina"

# #from twitterapistuff import *

# class listener(StreamListener):
#   def on_data(self, data):
#     all_data = json.loads(data)
#     tweet = all_data["text"]
#     sentiment_value, confidence = s.sentiment(tweet)
#     print(tweet, sentiment_value)
#     if sentiment_value=="pos":
#       output = open("twitter_pos_out.txt","a")
#       output.write(tweet)
#       output.write('\n')
#       output.close()
#     else:
#       output = open("twitter_neg_out.txt","a")
#       output.write(tweet)
#       output.write('\n')
#       output.close()
#     return True

#   def on_error(self, status):
#     print(status)

# auth = OAuthHandler(ckey, csecret)
# auth.set_access_token(atoken, asecret)

# twitterStream = Stream(auth, listener())
# twitterStream.filter(track=["rajini"])

Code switching

In [ ]:
pip install langdetect

     |████████████████████████████████| 983kB 10.6MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=3c4dda5642c801ee2d5aa6f5f005ed9c3da8bd0ee528022aefad2e63fd192581
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
Successfully built langdetect


In [ ]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 1.2MB 22.1MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=b539242428d4bf29b20e00e45f6f6cc949100996192f58a8ddf065083a164be7
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [ ]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())

In [ ]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())

In [ ]:
SGDC_classifier = SklearnClassifier(SGDClassifier())

In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/Research/NLTK VIthu (2).xlsx')

In [ ]:
#Cleaning the text
def tokenizer(sentence):
    mytokens=word_tokenize(sentence)
    return mytokens

In [ ]:
#Cleaning text
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [ ]:
vectorizer = CountVectorizer(tokenizer = tokenizer, ngram_range=(1,1)) 
tfvectorizer = TfidfVectorizer(tokenizer = tokenizer)

In [ ]:
X = dataset['Text']
y = dataset['Language']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [ ]:
#Using third model
SVCclassifier = LinearSVC()
SVCmodel = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', SVCclassifier)])

# Train the Model
SVCmodel.fit(X_train,y_train)   
pred3= SVCmodel.predict(X_test)
print(f'Confusion Matrix:\n{confusion_matrix(y_test,pred3)}')
print(f'\nClassification Report:\n{classification_report(y_test,pred3)}')
print(f'Accuracy: {accuracy_score(y_test,pred3)*100}%')

Confusion Matrix:
[[296   1   0]
 [  0 309   0]
 [  4  11 279]]

Classification Report:
              precision    recall  f1-score   support

     English       0.99      1.00      0.99       297
       Tamil       0.96      1.00      0.98       309
   Thanglish       1.00      0.95      0.97       294

    accuracy                           0.98       900
   macro avg       0.98      0.98      0.98       900
weighted avg       0.98      0.98      0.98       900

Accuracy: 98.22222222222223%


In [ ]:
BernoulliNB = BernoulliNB()
BernoulliNB_model = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', BernoulliNB)])

# Train the Model
BernoulliNB_model.fit(X_train,y_train)   
pred3= BernoulliNB_model.predict(X_test)
print(f'Confusion Matrix:\n{confusion_matrix(y_test,pred3)}')
print(f'\nClassification Report:\n{classification_report(y_test,pred3)}')
print(f'Accuracy: {accuracy_score(y_test,pred3)*100}%')

Confusion Matrix:
[[265   3  29]
 [  0 309   0]
 [  0   2 292]]

Classification Report:
              precision    recall  f1-score   support

     English       1.00      0.89      0.94       297
       Tamil       0.98      1.00      0.99       309
   Thanglish       0.91      0.99      0.95       294

    accuracy                           0.96       900
   macro avg       0.96      0.96      0.96       900
weighted avg       0.97      0.96      0.96       900

Accuracy: 96.22222222222221%


In [ ]:
logesticRegression = LogisticRegression()
logisticModel = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', logesticRegression)])
# Train the Model
logisticModel.fit(X_train,y_train)   
pred3= logisticModel.predict(X_test)
print(f'Confusion Matrix:\n{confusion_matrix(y_test,pred3)}')
print(f'\nClassification Report:\n{classification_report(y_test,pred3)}')
print(f'Accuracy: {accuracy_score(y_test,pred3)*100}%')


Confusion Matrix:
[[291   4   2]
 [  0 308   1]
 [  4  13 277]]

Classification Report:
              precision    recall  f1-score   support

     English       0.99      0.98      0.98       297
       Tamil       0.95      1.00      0.97       309
   Thanglish       0.99      0.94      0.97       294

    accuracy                           0.97       900
   macro avg       0.97      0.97      0.97       900
weighted avg       0.97      0.97      0.97       900

Accuracy: 97.33333333333334%


In [ ]:
SGDClassifier = SGDClassifier()
SGDV_Model = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', SGDClassifier)])
# Train the Model
SGDV_Model.fit(X_train,y_train)   
pred3= SGDV_Model.predict(X_test)
print(f'Confusion Matrix:\n{confusion_matrix(y_test,pred3)}')
print(f'\nClassification Report:\n{classification_report(y_test,pred3)}')
print(f'Accuracy: {accuracy_score(y_test,pred3)*100}%')

Confusion Matrix:
[[294   2   1]
 [  0 308   1]
 [  3  11 280]]

Classification Report:
              precision    recall  f1-score   support

     English       0.99      0.99      0.99       297
       Tamil       0.96      1.00      0.98       309
   Thanglish       0.99      0.95      0.97       294

    accuracy                           0.98       900
   macro avg       0.98      0.98      0.98       900
weighted avg       0.98      0.98      0.98       900

Accuracy: 98.0%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pre = SVCmodel.predict(["யாராவது முழு சொற்றொடரையும் மீண்டும் மீண்டும் செய்ய விரும்பினால் கடைசி வார்த்தையை நான் பிடிக்கவில்லை என்பதால் நீங்கள் அதை மீண்டும் செய்ய முடியுமா?"])
print(pre[0])

Tamil


In [ ]:
pre = SGDV_Model.predict(["யாராவது முழு சொற்றொடரையும் மீண்டும் மீண்டும் செய்ய விரும்பினால் கடைசி வார்த்தையை நான் பிடிக்கவில்லை என்பதால் நீங்கள் அதை மீண்டும் செய்ய முடியுமா?"])
print(pre[0])

Tamil


In [ ]:
t1="பத்த வச்ச நெருப்பு பரபரன்னு பற்றி எரிவது போல படம் ஓடுகிறது.. தொடக்கத்திலிருந்து இறுதி வரை."

In [ ]:
from langdetect import detect
from googletrans import Translator

print(detect(t1))
translator= Translator()
translatedSent= translator.translate(t1, src='ta', dest='en',)
print(translatedSent.text)

ta
The film runs like wildfire about the Patha Vachcha fire sensation .. from beginning to end.


Language detection and translation normalization

In [ ]:
#from langdetect import detect
from googletrans import Translator

tweet_try=["vanakam thala padam vera level la irunthathu msg "]


pre = SVCmodel.predict(tweet_try)
print(pre[0])
if pre[0]=="English":
  print(tweet_try[0])
elif pre[0]=="Thanglish":
  translator1= Translator()
  translatedSent= translator1.translate(tweet_try[0], src='ta', dest='en')
  print(translatedSent.text)  
else:
  translator= Translator()
  translatedSent= translator.translate(tweet_try[0],src='ta',dest='en')
  print(translatedSent.text) 

Thanglish
Hello head movie was on another level


In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sent_mod1 as s

#consumer key, consumer secret, access token, access secret.
ckey="XXeG5cy7QUfahiOHlacBaddIF"
csecret="quSrqvVZUfRqbp8SZ0bDZimz8avlS96htsonlG1dFAehgsmMI1"
atoken="1399846090699067392-W14id1VCFiLIib3MD7kaT2C6pvDmvX"
asecret="6nrOLFSELerY9HzGSV5YGNigZnhSM54izskj2iahnFina"

#from twitterapistuff import *

class listener(StreamListener):
  def on_data(self, data):
    all_data = json.loads(data)
    tweet = all_data["text"]
    tweet_try=[tweet]
    pre = SVCmodel.predict(tweet_try)
    if pre[0]=="English":
      sentiment_value, confidence = s.sentiment(tweet)
    elif pre[0]=="Thanglish":
      translator1= Translator()
      translatedSent= translator1.translate(tweet_try[0], src='ta', dest='en')
      sentiment_value, confidence = s.sentiment('translatedSent')  
    else:
      translator= Translator()
      translatedSent= translator.translate(tweet_try[0], src='ta', dest='en')
      sentiment_value, confidence = s.sentiment('translatedSent')
    # sentiment_value, confidence = s.sentiment(tweet)
    print(tweet, sentiment_value)
    if sentiment_value=="pos":
      output = open("twitter_pos_out.txt","a")
      output.write(tweet)
      output.write('\n')
      output.close()
    else:
      output = open("twitter_neg_out.txt","a")
      output.write(tweet)
      output.write('\n')
      output.close()
    return True

  def on_error(self, status):
    print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["valimai"])

ModuleNotFoundError: ignored